In [2]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('RFM_ht_data.zip', parse_dates=['InvoiceDate'], dtype={"InvoiceNo": str, "CustomerCode": str, "Amount": "float64"})

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     332730 non-null  object        
 1   CustomerCode  332730 non-null  object        
 2   InvoiceDate   332730 non-null  datetime64[ns]
 3   Amount        332730 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 10.2+ MB


In [10]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [52]:
last_date = df.InvoiceDate.max()
last_date

Timestamp('2020-09-30 00:00:00')

In [54]:
rfm_df = df.groupby(by='CustomerCode', as_index=False) \
    .agg(
        {
            'InvoiceDate': lambda x: (last_date - x.max()).days, # Recency 
            'InvoiceNo': len, # Frequency
            'Amount': 'sum' # Monetary
        } 
    )

In [55]:

rfm_df = rfm_df.rename(columns={
    'InvoiceDate':'Recency',
    'InvoiceNo' : 'Frequency',
    'Amount':'Monetary'})

rfm_df.head()

,CustomerCode,Recency,Frequency,Monetary
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88


In [64]:
def RClass(value, parameter_name, quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FMClass(value, parameter_name, quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1

In [66]:
df_quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
df_quantiles

,Recency,Frequency,Monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [68]:
rfm_df['r'] = rfm_df.Recency.apply(RClass, args=('Recency', df_quantiles))
rfm_df['f'] = rfm_df.Frequency.apply(FMClass, args=('Frequency', df_quantiles))
rfm_df['m'] = rfm_df.Monetary.apply(FMClass, args=('Monetary', df_quantiles))

,CustomerCode,Recency,Frequency,Monetary,r,f,m
0,02213019,19,1,1609.20,4,4,3
1,02213042,22,3,9685.48,4,2,1
2,02213071,29,1,415.00,4,4,4
3,02213088,23,1,305.00,4,4,4
4,02213092,25,1,1412.88,4,4,3


In [69]:
rfm_df['RFMClass'] = rfm_df.r.map(str) + rfm_df.f.map(str) + rfm_df.m.map(str)

rfm_df.head()

,CustomerCode,Recency,Frequency,Monetary,r,f,m,RFMClass
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443


## Задание 2
Какое максимальное кол-во покупок было совершено одним пользователем?

In [17]:
df_invoices = df.groupby(by='CustomerCode', as_index=False) \
    .agg({'InvoiceNo':'count'}) \
    .sort_values(by='InvoiceNo', ascending=False) \
    .rename(columns={'InvoiceNo':'Invoice_counts'})

df_invoices.head()

,CustomerCode,Invoice_counts
89388,19057820,204
44594,13215452,113
10347,13032521,106
97077,19080880,99
119951,99003061,90


## Задание 3
Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)

In [59]:
rfm_df.Monetary.quantile(q=[0.25,0.5,0.75])

0.25     765.00
0.50    1834.48
0.75    4008.84
Name: Monetary, dtype: float64

1. (4008.84:~]
2. (1834.48:4008.84]
3. (765.00:1834.48]
4. `(0:765.00]`

## Задание 4
Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?

In [60]:
rfm_df.Frequency.quantile(q=[0.25,0.5,0.75])

0.25    1.0
0.50    2.0
0.75    3.0
Name: Frequency, dtype: float64

1. `(3:~]`
2. (2:3]
3. (1:2]
4. (0:1]

In [63]:
rfm_df.query('Frequency > 3').min()

CustomerCode    02213129
Recency                0
Frequency              4
Monetary           265.0
dtype: object

## Задание 5
Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R?

In [58]:
rfm_df.Recency.quantile(q=[0.25,0.5,0.75])

0.25     2.0
0.50     8.0
0.75    16.0
Name: Recency, dtype: float64

1. (0:2]
2. `(2:8]`
3. (8:16]
4. (16:~]

## Задание 6
Сколько пользователей попало в сегмент 111?

In [75]:
rfm_df_count = rfm_df \
    .groupby(by='RFMClass') \
    .agg({'CustomerCode':'count'}) \
    .rename(columns={'CustomerCode':'Customers_counts'}) \
    .sort_values(by='Customers_counts', ascending=False)

rfm_df_count.loc['111']

Customers_counts    9705
Name: 111, dtype: int64

## Задание 7
Сколько пользователей попало в сегмент 311?

In [74]:
rfm_df_count.loc['311']

Customers_counts    1609
Name: 311, dtype: int64

## Задание 8
В каком RFM-сегменте самое большое кол-во пользователей?

In [76]:
rfm_df_count.head()

,Customers_counts
RFMClass,
444,10624
111,9705
443,6729
344,6593
211,5847


## Задание 9
В каком RFM-сегменте самое маленькое кол-во пользователей?

In [81]:
rfm_df_count.tail()

,Customers_counts
RFMClass,
424,63
214,60
114,60
314,33
414,2


## Задание 10
Какое количество пользователей попало в самый малочисленный сегмент?

In [82]:
rfm_df_count.tail()

,Customers_counts
RFMClass,
424,63
214,60
114,60
314,33
414,2
